In [89]:
import pandas as pd
import requests as req
import json
import datetime
import warnings
import time as tm
warnings.filterwarnings("ignore")

In [6]:
r = req.get("https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2023-06-28/2023-06-28?adjusted=true&sort=asc&limit=5000&apiKey=5msoPvKD_Vl1it_uorssQeitM0MnHdYF")
res = json.loads(r.content)
df = pd.DataFrame(res)

In [7]:
df1 = df[['ticker']]
df1['OpenPrice'] = df['results'].apply(lambda x: x.get('o'))
df1['ClosePrice'] = df['results'].apply(lambda x: x.get('c'))
df1['Highp'] = df['results'].apply(lambda x: x.get('h'))
df1['Lowp'] = df['results'].apply(lambda x: x.get('l'))
df1['Volume'] = df['results'].apply(lambda x: x.get('v'))
df1['VWAP'] = df['results'].apply(lambda x: x.get('vw'))
df1['Time'] = df.results.apply(lambda x: datetime.datetime.fromtimestamp(int(x.get('t')) / 1000))
df1['Date'] = pd.to_datetime(df1['Time']).dt.date
df1['Time'] = pd.to_datetime(df1['Time']).dt.time

In [8]:
df1['Time'] = pd.to_datetime(df1['Time'], format='%H:%M:%S')
df1 = df1[(df1['Time'].dt.time > pd.to_datetime('16:25:00', format='%H:%M:%S').time()) & (df1['Time'].dt.time < pd.to_datetime('23:00:00', format='%H:%M:%S').time())]
df1

,ticker,OpenPrice,ClosePrice,Highp,Lowp,Volume,VWAP,Time,Date
56,AAPL,187.930,188.0900,188.310,187.710,1748159.0,188.0297,1900-01-01 16:30:00,2023-06-28
57,AAPL,188.080,188.2800,188.499,188.020,987922.0,188.2801,1900-01-01 16:35:00,2023-06-28
58,AAPL,188.265,188.3300,188.630,188.140,1031623.0,188.3852,1900-01-01 16:40:00,2023-06-28
59,AAPL,188.295,188.2692,188.350,188.010,910888.0,188.1730,1900-01-01 16:45:00,2023-06-28
60,AAPL,188.260,187.6050,188.310,187.600,874893.0,187.9567,1900-01-01 16:50:00,2023-06-28
...,...,...,...,...,...,...,...,...,...
129,AAPL,188.980,188.9000,189.020,188.850,327783.0,188.9230,1900-01-01 22:35:00,2023-06-28
130,AAPL,188.895,188.7101,189.050,188.685,476554.0,188.8690,1900-01-01 22:40:00,2023-06-28
131,AAPL,188.710,188.7800,188.900,188.690,500335.0,188.8216,1900-01-01 22:45:00,2023-06-28
132,AAPL,188.790,188.8048,189.060,188.620,924903.0,188.8720,1900-01-01 22:50:00,2023-06-28


In [79]:
df2 = df1.head(1)
df2.drop(df2.index, inplace=True)
df2

,ticker,OpenPrice,ClosePrice,Highp,Lowp,Volume,VWAP,Time,Date


In [ ]:
https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2023-06-28/2023-06-28?adjusted=true&sort=asc&limit=5000&apiKey=5msoPvKD_Vl1it_uorssQeitM0MnHdYF

In [86]:
Request_Date = '2023-06-01'  #start date
Req_per_minute = 4 #can request using the api up to 5 times per minute, i went with 4
delay = 60 / Req_per_minute
Total_Req = 30

for i in range(Total_Req):
    #check to avoid weekends
    year1, month1, day1 = Request_Date.split('-')  
    year1 = int(year1); month1 = int(month1); day1 = int(day1);
    if datetime.date(year1,month1,day1).strftime('%A') == 'Saturday':
        Request_Date = (datetime.datetime.strptime(Request_Date, '%Y-%m-%d') + datetime.timedelta(days=2)).strftime('%Y-%m-%d')

    #Requesting data    
    Request_string = 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/'+Request_Date+'/'+Request_Date+'?adjusted=true&sort=asc&limit=5000&apiKey=5msoPvKD_Vl1it_uorssQeitM0MnHdYF'
    r = req.get(Request_string)
    res = json.loads(r.content)
    df = pd.DataFrame(res)
    #Clean/Organize
    df1 = df[['ticker']]
    df1['OpenPrice'] = df['results'].apply(lambda x: x.get('o'))
    df1['ClosePrice'] = df['results'].apply(lambda x: x.get('c'))
    df1['Highp'] = df['results'].apply(lambda x: x.get('h'))
    df1['Lowp'] = df['results'].apply(lambda x: x.get('l'))
    df1['Volume'] = df['results'].apply(lambda x: x.get('v'))
    df1['VWAP'] = df['results'].apply(lambda x: x.get('vw'))
    df1['Time'] = df.results.apply(lambda x: datetime.datetime.fromtimestamp(int(x.get('t')) / 1000))
    df1['Date'] = pd.to_datetime(df1['Time']).dt.date
    df1['Time'] = pd.to_datetime(df1['Time']).dt.time
    #Filter only the standard trading time (Israel)
    df1['Time'] = pd.to_datetime(df1['Time'], format='%H:%M:%S')
    df1 = df1[(df1['Time'].dt.time > pd.to_datetime('16:25:00', format='%H:%M:%S').time()) & (df1['Time'].dt.time < pd.to_datetime('23:00:00', format='%H:%M:%S').time())]

    #concating the requests together to one big table (pre-made an empty df2)
    df2 = pd.concat([df1, df2])
    #update the date forward
    Request_Date = (datetime.datetime.strptime(Request_Date, '%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    #Delay 
    tm.sleep(delay)
    if i + 1 == Total_Req:
        break   

In [90]:

Req_per_minute = 4
Total_Req = 30
delay = 60 / Req_per_minute

# Run the code for the specified number of iterations
for i in range(Total_Req):
    # Your code goes here
   
    tm.sleep(delay)
    if i + 1 == Total_Req:
        break
    


Running code...
Running code...


KeyboardInterrupt: 

In [87]:
df2

,ticker,OpenPrice,ClosePrice,Highp,Lowp,Volume,VWAP,Time,Date
66,AAPL,182.6300,183.3050,183.4600,181.97,5140672.0,182.6732,1900-01-01 16:30:00,2023-06-05
67,AAPL,183.4004,183.5500,183.7584,183.21,1995551.0,183.4746,1900-01-01 16:35:00,2023-06-05
68,AAPL,183.5600,183.6600,183.7200,183.43,1304094.0,183.5841,1900-01-01 16:40:00,2023-06-05
69,AAPL,183.6550,183.7300,183.8600,183.57,1630265.0,183.7326,1900-01-01 16:45:00,2023-06-05
70,AAPL,183.7300,182.8486,183.7600,182.71,1812181.0,183.1838,1900-01-01 16:50:00,2023-06-05
...,...,...,...,...,...,...,...,...,...
134,AAPL,179.6100,179.3250,179.6200,179.22,902948.0,179.4105,1900-01-01 22:35:00,2023-06-01
135,AAPL,179.3250,179.4700,179.6650,179.25,966980.0,179.5101,1900-01-01 22:40:00,2023-06-01
136,AAPL,179.4650,179.7500,179.7700,179.44,926151.0,179.5953,1900-01-01 22:45:00,2023-06-01
137,AAPL,179.7400,179.8050,179.9400,179.46,1458404.0,179.6442,1900-01-01 22:50:00,2023-06-01


In [88]:
df2.Date.unique()

array([datetime.date(2023, 6, 5), datetime.date(2023, 6, 2),
       datetime.date(2023, 6, 1)], dtype=object)